# Generate Synthetic text summarization with Gretel GPT

* In this notebook we use Gretel GPT with Llama-2 7b model to create synthetic text summerization dataset.  
* To run this notebook, you will need an API key from the [Gretel Console](https://console.gretel.ai/users/me/key/).

## Getting Started

In [ ]:
%%capture
!pip install -U gretel-client

In [ ]:
import pandas as pd

from gretel_client import configure_session
from gretel_client.helpers import poll
from gretel_client.projects import create_or_get_unique_project, get_project

In [ ]:
# Log into Gretel
configure_session(api_key="prompt", cache="yes", endpoint="https://api.gretel.cloud", validate=True, clear=True)

pd.set_option('max_colwidth', None)

## Load and preview training data

In [ ]:
# Specify a dataset to train on
DATASET_PATH = 'https://gretel-datasets.s3.us-west-2.amazonaws.com/Text-dataset/Samsum-text-summerization-sample-1000.csv'
df = pd.read_csv(DATASET_PATH)

df.head()

## Configure and Train the Synthetic Model:

We can experiment different "steps" parameters which result in a change of text SQS.

In [ ]:
from gretel_client.projects.models import read_model_config



config = read_model_config("synthetics/natural-language")
config["models"][0]["gpt_x"]["steps"] = 1000 #set different step values.

# Designate project
PROJECT = 'data-summarization'
project = create_or_get_unique_project(name=PROJECT)

# Create and submit model
model = project.create_model_obj(model_config=config, data_source=df)
model.name = f"{PROJECT}-llama-2-7b"
model.submit_cloud()

poll(model)


## Generate Text Synthetic Quality Score:

In [ ]:
model.get_report_summary()

In [ ]:
#Plot the text SQS report:
import IPython
from smart_open import open

IPython.display.HTML(data=open(model.get_artifact_link("text_metrics_report")).read(), metadata=dict(isolated=True))